In [ ]:
import pandas as pd

from portfawn.market_data import MarketData
from portfawn.portfolio import Portfolio

%matplotlib inline

In [ ]:
## Parameters
# market data
tickers = {"NASDAQ": "^IXIC", 
              "RUSSELL2000": "^RUT", 
              "S&P500": "^GSPC", 
              "HANG SENG INDEX": "^HSI"}
asset_list = list(tickers.values())

# date
start_date = "2000-01-01"
end_date   = "2020-12-30"

# portfolio
portfolio_fitness = "Equal"
risk_free_rate = 0.0
sampling_params = {"type": "standard"}
optimization_params = {
    "scipy_params": {
        "maxiter": 1000,
        "disp": False,
        "ftol": 1e-10,
    },
    "target_return": 0.1,
    "target_risk": 0.1,
    "weight_bound": (0.0, 1.0),
}

In [ ]:
## Create the market instance
market_data = MarketData(
    tickers = tickers,
    date_start = start_date,
    date_end = end_date,
    col_price = "Close",
)
data_returns = market_data.data_returns

In [ ]:
portfolio = Portfolio(
        portfolio_fitness=portfolio_fitness,
        data_returns=data_returns,
        risk_free_rate=risk_free_rate,
        optimization_params=optimization_params,
        sampling_params=sampling_params,
    )
portfolio.optimize()
portfolio.evaluate()

In [ ]:
# fig, ax = portfolio.plot_returns()

In [ ]:
# fig, ax = portfolio.plot_cum_returns()

In [ ]:
# fig, ax = portfolio.plot_dist_returns()

In [ ]:
# fig, ax = portfolio.plot_corr()

In [ ]:
# fig, ax = portfolio.plot_cov()

In [ ]:
# fig, ax = portfolio.plot_mean_std()

In [ ]:
import numpy as np
from portfawn.plot import Plot

plot = Plot()


portfolio_fitness_list = ["Equal", "MSR", "MR", "MV"]
mean_std_list = []

for portfolio_fitness in portfolio_fitness_list:
    portfolio = Portfolio(
        portfolio_fitness=portfolio_fitness,
        data_returns=portfolio.data_returns,
    )

    portfolio.optimize()
    portfolio.evaluate()
    mean_std_list.append(portfolio.performance["portfolio_mean_std"])

market_mean_std = portfolio.performance["market_mean_std"]
portfolio_mean_std = pd.concat(mean_std_list, axis=0)

market_mean_std["mean"] *= 252
market_mean_std["std"] *= np.sqrt(252)
portfolio_mean_std["mean"] *= 252
portfolio_mean_std["std"] *= np.sqrt(252)

In [ ]:
n = 100000
annualized = 252
returns_np = portfolio.data_returns.to_numpy()
cov = portfolio.data_returns.cov().to_numpy()
r_list = []
for i in range(n):
    w_rand = np.random.random((1, cov.shape[0]))
    w_rand = w_rand / w_rand.sum()
    r = returns_np.dot(w_rand.T).mean() * annualized
    c = np.sqrt(w_rand.dot(cov).dot(w_rand.T))[0][0] * np.sqrt(annualized)
    r_list.append({"mean": r, "std": c})
mean_std_random = pd.DataFrame(r_list)

In [ ]:

fig, ax = plot.plot_scatter_portfolio_random(
    df_1=market_mean_std,
    df_2=portfolio_mean_std,
    df_3=mean_std_random,
    title="Expected Returns vs. Volatility",
    xlabel="Volatility (STD)",
    ylabel="Expected Returns",
)
